In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://1c173c8c3406:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 01:58:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from grandeInformacion.spark_utils import SparkUtils
from pyspark.sql.functions import col, when, count, isnull
netflix_schema = SparkUtils.generate_schema([("show_id", "string"), ("type", "string"), ("title", "string"), ("director", "string"), ("country", "string"), ("date_added", "string"), ("release_year", "string"), \
                                            ("rating", "string"), ("duration", "string"), ("listed_in", "string")])

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            

In [4]:

netflix_df.createOrReplaceTempView('netflix')

netflix_columns = ["show_id","type","title","director","country","date_added","release_year","rating","duration","listed_in"]
netflix_df.select([count(when(isnull(c) | col(c).isNull(), c)).alias(c) for c in netflix_columns]).show()

try_df = SparkUtils.clean_df(netflix_df)
try_df.createOrReplaceTempView('try')

netflix_columns = ["show_id","type","title","director","country","date_added","release_year","rating","duration","listed_in"]
try_df.select([count(when(isnull(c) | col(c).isNull(), c)).alias(c) for c in netflix_columns]).show()

SparkUtils.write_df(try_df)

+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       1|      1|         1|           1|     1|       2|        2|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+

+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       0|      0|         0|           0|     0|       0|        0|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



In [5]:
!ls notebooks/data/netflix_output/ | wc -l

75
